# Investigating Neural Networks

We'll be looking at different networks to discuss their differences, advantages, and what they look like. For this we will use the `torchvision` and `timm` packages from python as these pacakges have a substantial amount of vision models and are the de facto places one should look for when vision models


In [4]:
%load_ext tensorboard

In [5]:
import torch
import torchvision
from torchvision import transforms as T
import timm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np

In [6]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [7]:
transform = T.ToTensor()
trainset = torchvision.datasets.CIFAR10('../Datasets', download=True, transform=transform)
train = torch.utils.data.DataLoader(trainset, batch_size=32, num_workers=12)

expand_transform = T.Compose([T.Resize(256), T.ToTensor()])
trainset_256 = torchvision.datasets.CIFAR10('../Datasets', download=True, transform=expand_transform)
train_256 = torch.utils.data.DataLoader(trainset_256, batch_size=32, num_workers=12)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../Datasets\cifar-10-python.tar.gz to ../Datasets


C:\Users\bbrad\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [ ]:
dataiter = iter(train)
imgs, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(imgs)
matplotlib_imshow(img_grid)

dataiter_256 = iter(train_256)
imgs_256, labels_256 = dataiter_256.next()
img_grid_256 = torchvision.utils.make_grid(imgs_256)
matplotlib_imshow(img_grid_256)

# LeNet

In [ ]:
from torch import nn
from torch.nn import functional as F

class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=389880, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        print(x.shape)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [ ]:
lenet = LeNet5(10)
lewriter = SummaryWriter("lenet")
lewriter.add_image('cifar', img_grid_256)
lewriter.add_graph(lenet, imgs_256)
%tensorboard --logdir lenet

# AlexNet

In [ ]:
alex = torchvision.models.alexnet(pretrained=True)
alex_writer = SummaryWriter("alex")
alex_writer.add_image('cifar', img_grid_256)
alex_writer.add_graph(alex, imgs_256)

%tensorboard --logdir alex

# VGG19

In [ ]:
vgg = torchvision.models.vgg19(pretrained=True)
vgg_writer = SummaryWriter("vgg")
vgg_writer.add_image('cifar', img_grid_256)
vgg_writer.add_graph(vgg, imgs_256)

%tensorboard --logdir vgg

# GoogLeNet

In [ ]:
google = torchvision.models.googlenet()
goog_writer = SummaryWriter("google")
goog_writer.add_image('cifar', img_grid_256)
goog_writer.add_graph(google, imgs_256)

%tensorboard --logdir google

# ResNet34

In [ ]:
resnet = timm.create_model('resnet34')
res_writer = SummaryWriter("resnet")

res_writer.add_image('cifar', img_grid)


res_writer.add_graph(resnet, imgs)

%tensorboard --logdir resnet34

# MobileNet

In [ ]:
mobile = timm.create_model('mobilenetv3_large_075', pretrained=True)
mob_writer = SummaryWriter("mobilenet")
mob_writer.add_image('cifar', img_grid_256)
mob_writer.add_graph(mobile, imgs_256)

%tensorboard --logdir mobilenet

# Looking at Answers

In [ ]:
vgg

In [ ]:
# VGG was trained on ImageNet, but let's use CIFAR-10
vgg.classifier[6] = nn.Linear(4096, 10)
vgg.classifier

In [ ]:
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 12))
    for idx in np.arange(16):
        ax = fig.add_subplot(4, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx])
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [ ]:

vgg_writer.add_figure('predictions vs actual',
                     plot_classes_preds(vgg, imgs_256, labels_256),
                     global_step=1
                     )

In [ ]:
%tensorboard --logdir vgg